## Demand Prediction Model using Time Series

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import calendar
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df_initial = pd.read_pickle('./DemandDataFile', compression='infer')
df_region = pd.read_pickle('./RegionDataFile', compression='infer')
df_initial = pd.merge(df_initial, df_region, how='inner', right_on=['CITY_NAME'], left_on=['CITY'])
df_initial = df_initial.drop(['CITY_NAME'], axis=1)
df_initial = df_initial[~df_initial['PRODUCT_NAME'].str.contains("Small Flyers|Large Flyers|Meter Bubble Wrap|Bundle of 50 Boxes", na=False)]

In [ ]:
df_initial.rename(columns = {'ORDER_DATE':'DATE'},inplace = True)

In [ ]:
df_initial.head()

In [ ]:
df_weights = pd.read_csv('./ProductWeights.csv')
df_weights.drop_duplicates(subset=['COD_SKU_CONFIG'],inplace =True)

df_productReviews = pd.read_csv('./ProductReviews.csv')
df_productReviews.drop_duplicates(subset=['COD_SKU_CONFIG'],inplace =True)

In [ ]:
df_weights.columns = ['SKU','PRODUCT_NAME','WEIGHT']
df_initial = pd.merge(df_initial, df_weights[['SKU','WEIGHT']], how='left')

In [ ]:
df_productReviews.columns = ['SKU','AVG_RATING']
df_initial = pd.merge(df_initial, df_productReviews, how='left')

In [ ]:
df_initial.loc[:,"Voucher"][df_initial.Voucher > 0] = True
df_initial.loc[:,"Voucher"][df_initial.Voucher != True] = False

### Add Month & WeekDay Columns 

In [ ]:
df_initial['WEEKDAY'] = df_initial['DATE'].apply(lambda x:calendar.day_name[x.weekday()])
df_initial['MONTH'] = df_initial['DATE'].apply(lambda x:calendar.month_abbr[x.month])


In [ ]:
tab_info = pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.rename(index={0:'null values (%)'}))
display(tab_info)

### Removing SKUs with least History

In [ ]:
totalDays = 180 
MinHistoryDays = int(totalDays * 0.02)
#print(MinHistoryDays)
temp = df_initial.groupby(['SKU'])['DATE'].count().to_frame('Count').reset_index()
temp = temp[temp.Count >= MinHistoryDays]

df_initial = pd.merge(df_initial, temp, how='inner')
len(temp)

### WEIGHT Column Cleaning 

In [ ]:
import re
df_initial['WEIGHT'] = df_initial['WEIGHT'].apply(lambda x : re.sub(r'#', r'0', str(x)))
df_initial['WEIGHT'] = df_initial['WEIGHT'].apply(lambda x : re.sub(r'[a-zA-Z]', '', str(x)))
df_initial['WEIGHT'] = df_initial['WEIGHT'].apply(lambda x : re.sub(r'([.])\1+', r'\1', str(x)))
df_initial['WEIGHT'] = df_initial['WEIGHT'].apply(lambda x : re.sub(r'/', r'', str(x)))
df_initial['WEIGHT'] = df_initial['WEIGHT'].apply(lambda x : re.sub(r'[!]', '0', str(x)))
df_initial['WEIGHT'] = pd.to_numeric(df_initial['WEIGHT'])
weights_temp = df_initial['WEIGHT']

df_initial.WEIGHT[df_initial['WEIGHT'].isnull()] = -1
df_initial.WEIGHT[df_initial['WEIGHT'] >= 100] = df_initial.WEIGHT[df_initial['WEIGHT'] >= 100] / 1000
len(df_initial.WEIGHT[df_initial['WEIGHT'] >= 100] / 1000)
bins = [-2 , -1 ,0 , 10, 20, 30 ,40 ,50 ,60 ,70 ,df_initial['WEIGHT'].max()]
labels = ['Unknown', 'Low < 0','Low (>0 <10)','Low (>10 <20)','Med (>20 <30)', 
          'Med (>30 <40)','Med (>40 <50)','Hi (>50 <60)','Hi (>60 <70)','Highest (>70)']
df_initial['WEIGHT_BINNED'] = pd.cut(df_initial['WEIGHT'], bins=bins, labels=labels)

### UNIT PRICE Column Binning / Filtring 

In [ ]:
df_initial['PRICE_MEDIAN'] = df_initial.groupby('SKU')['UNIT_PRICE'].transform('median')
bins = [1 , 15000 ,45000 , 100000 ,150000 ,df_initial['PRICE_MEDIAN'].max()]
df_initial['PRICE_BINNED'] = pd.cut(df_initial['PRICE_MEDIAN'],bins=bins, labels=["low","lower medium", "medium","higher medium", "expensive"])
df_initial['PRICE_BINNED'].value_counts()

### SKU PRICE mapping check 

In [ ]:
temp = df_initial.groupby(['SKU'])['PRICE_MEDIAN'].nunique()  > 1
if len(temp[temp == False]) != df_initial['SKU'].nunique():
       print ('Some SKUs have duplicate prices, Should resolve this issue before resuming')
else:
       print('All SKUs in dataset have SINGLE price associated, we can resume')


### Promotions Dates Data 

In [ ]:
df_promotions = pd.read_csv('./PromotionDates.csv')

df_promotions.StartDate = pd.to_datetime(df_promotions['StartDate'])
df_promotions.EndDate = pd.to_datetime(df_promotions['EndDate'])
df_promotions['Days'] = (df_promotions['EndDate'] - df_promotions['StartDate']).dt.days + 1
#repeat rows
df_promotions = df_promotions.loc[df_promotions.index.repeat(df_promotions['Days'])]
#group by index with transform for date ranges
df_promotions['Dates'] =(df_promotions.groupby(level=0)['StartDate']
                         .transform(lambda x: pd.date_range(start=x.iat[0], periods=len(x))))
#unique default index
df_promotions = df_promotions.reset_index(drop=True)


### Region to Warehouse mapping 

In [ ]:
df_initial['WareHouse'] = 'Null'

In [ ]:
df_initial.loc[:,"WareHouse"][df_initial['REGION_NAME'].isin(['Sindh','Balochistan'])] = 'Karachi'
df_initial.loc[:,"WareHouse"][~df_initial['REGION_NAME'].isin(['Sindh','Balochistan'])] = 'Lahore'

In [ ]:
df_initial['WareHouse'].value_counts()

## DataFrame for ARIMA Time Series Model

In [ ]:
df_Model = df_initial[['SKU','PRODUCT_NAME','DATE','Quantity','WareHouse']]

In [ ]:
df_Model = df_Model.groupby(by=['SKU','DATE'], as_index=False)['Quantity'].sum()
df_Model.sort_values('DATE',ascending=True, inplace = True)
df_Model.DATE = pd.to_datetime(df_Model['DATE'])
df_Model = df_Model.set_index('DATE')

#### TS Vis to check if Trend , Seasonality exists (i.e Sationary or Non- Stationary)

In [ ]:
dataTS = df_Model[df_Model.SKU == 'SH069FA039PJONAFAMZ'][['Quantity']]
q = dataTS.Quantity.quantile(0.96)

dataTS.Quantity[dataTS.Quantity > dataTS.Quantity.quantile(0.96)] = int(q)

q = dataTS.Quantity.quantile(0.01)

dataTS.Quantity[dataTS.Quantity < dataTS.Quantity.quantile(0.01)] = int(q)

series = dataTS.copy()
series.plot()
series.hist()

#### Stationary Test using Simple Mean / Varience measures , spliting dataset into 2 halves

In [ ]:
X = series.values
split = len(X) / 2
X1, X2 = X[0:int(split)+1], X[int(split):]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('Mean1=%f, Mean2=%f' % (mean1, mean2))
print('Variance1=%f, Variance2=%f' % (var1, var2))

#### Augmented Dickey-Fuller test for Stationary Test

In [ ]:
from statsmodels.tsa.stattools import adfuller
X = series.values
result = adfuller(pd.DataFrame(X).iloc[:,0])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))


print('If ADF value is Negative and lesser then Critical values then TS is Stationary , else Non Stationary')

In [ ]:
#from pandas.tools.plotting import autocorrelation_plot
#autocorrelation_plot(series)
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(series,lags=40)
print('')
#series = np.array(series, dtype=np.float64)

### ARMIA Model Building

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(series.values, order=(4,0,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
from sklearn.metrics import mean_squared_error
import math

X = series.values
size = int(len(X) * 0.85)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(2,0,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    #print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)

tsModelErrors = {'ARIMA Model': math.sqrt(error)}

# plot
plt.figure(figsize=(16,8))
plt.suptitle('Time Series Prediction for (Pack of 2 - Black & Grey Cotton Tshirts for Men) for 27 Days', fontsize= 16)
plt.title('Test Unit Error: %.2f' % math.sqrt(error), fontsize=18)

plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

#Pack of 2 - Black & Grey Cotton Tshirts for Men

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

train_size = int(len(dataTS) * 0.85)
train, test = dataTS[0:train_size], dataTS[train_size:]
    
y_hat_avg = test.copy()

fit1 = ExponentialSmoothing(train['Quantity'].ravel(),seasonal='add',seasonal_periods=11).fit(remove_bias = True,optimized = True)
y_hat_avg['Holt_winter'] = fit1.forecast(len(test))
rms = math.sqrt(mean_squared_error(test.Quantity, y_hat_avg.Holt_winter))
tsModelErrors['Holt Winter'] = rms

plt.figure(figsize=(16,8))
#plt.plot(train['Quantity'], label='Train')
plt.suptitle('Time Series Prediction for (Pack of 2 - Black & Grey Cotton Tshirts for Men) for 27 Days', fontsize= 16)
plt.title('Test Unit Error: %.2f' % rms, fontsize=18)

plt.plot(test['Quantity'], label='Test')
plt.plot(y_hat_avg['Holt_winter'], label='Holt_winter')
plt.legend(loc='best')
plt.show()

#pd.concat([test.Quantity, y_hat_avg.Holt_winter], axis=1).reset_index()

In [ ]:
from fbprophet import Prophet
dataProphet = dataTS.reset_index()
dataProphet.columns = ['ds','y']
m = Prophet(changepoint_prior_scale=0.)
m.fit(dataProphet);

In [ ]:
future = m.make_future_dataframe(periods=10)
future.tail(10)

forecast = m.predict(future)
forecast['y'] = dataProphet.y
forecast.rename(columns={'ds': 'Date','y': 'Actual Demand','yhat': 'Prediction','yhat_lower': 'Lower Bound','yhat_upper': 'Upper Bound'}, inplace=True)
#forecast.columns = ['Date','Actual Demand','Prediction','Lower Bound','Upper Bound']
forecast[['Date','Actual Demand','Prediction','Lower Bound','Upper Bound']][161:180]

In [ ]:
#m.plot(forecast);
rms = math.sqrt(mean_squared_error(forecast['Actual Demand'][154:180], forecast.Prediction[154:180]))
tsModelErrors['Facebook Prophet Library'] = rms

plt.figure(figsize=(16,8))
#plt.plot(train['Quantity'], label='Train')
plt.suptitle('Time Series Prediction using FACEBOOK library for (Pack of 2 - Black & Grey Cotton Tshirts for Men) for 27 Days', fontsize= 16)
plt.title('Test Unit Error: %.2f' % rms, fontsize=18)

plt.plot(forecast['Actual Demand'][154:180], label='Test')
plt.plot(forecast['Prediction'][154:180], label='Predictions')
plt.legend(loc='best')
plt.show()

In [ ]:
m.plot_components(forecast);

In [ ]:
from matplotlib.ticker import FormatStrFormatter
fig, ax = plt.subplots(figsize=(7, 5))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.bar(range(len(tsModelErrors)), list(tsModelErrors.values()), align='center')
plt.xticks(range(len(tsModelErrors)), list(tsModelErrors.keys()))
plt.title('Error Comparision between the Time Series Models', fontsize=18)

rects = ax.patches
labels = ["Unit Error = %.2f" % i for i in (tsModelErrors.values())]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height, label,
            ha='center', va='bottom')


In [ ]:
import warnings
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.8)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
 

# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)